<a href="https://colab.research.google.com/github/algo21-221040114/PytorchTest/blob/main/PytorchLightning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split
from torch.utils.data import TensorDataset, DataLoader, random_split

In [ ]:
pip install pytorch_lightning

In [ ]:
pip install torchmetrics

In [4]:
from torchmetrics.functional import accuracy

import pytorch_lightning
from pytorch_lightning import LightningModule, Trainer

In [38]:
# Pytorch Lightning
# reference: https://zhuanlan.zhihu.com/p/410394574
class PLModel(LightningModule):
  
  def __init__(self, channels, height, width, hidden_size, batch_size):
    super().__init__()

    self.channels = channels
    self.height = height
    self.width = width
    self.hidden_size = hidden_size
    self.batch_size = batch_size

    self.model = nn.Sequential(
        nn.Flatten(),
        nn.Linear(self.channels * self.height * self.width, self.hidden_size*2),
        nn.ReLU(),
        nn.Dropout(0.1),
        nn.Linear(self.hidden_size*2, self.hidden_size),
        nn.ReLU(),
        nn.Dropout(0.1),
        nn.Linear(self.hidden_size, 1)
    )

  def forward(self, x):
    x = self.model(x)
    return x

  def configure_optimizers(self):
    optimizer = torch.optim.Adam(self.parameters(), lr=1e-2)
    return optimizer

  def training_step(self, batch, batch_idx):
    x, y = batch
    pred_y = self.model(x)
    loss_fn = nn.MSELoss(reduce=True, size_average=True)
    loss = loss_fn(pred_y.float(), y.float())
    return loss

  # def validation_step(self, batch, batch_idx):
  #   x, y = batch, batch_idx
  #   pred_y = self.model(x)
  #   loss = nn.MSELoss(pred_y, y)

  #   self.log('val_loss', loss, prog_bar=True)
  #   return loss
    
  # def test_step(self, batch, batch_idx):
  #     return self.validation_step(batch, batch_idx)

  # def train_dataloader(self):
  #     return DataLoader(self.mnist_train, batch_size=self.batch_size)

  # def val_dataloader(self):
  #     return DataLoader(self.mnist_val, batch_size=self.batch_size)

  # def test_dataloader(self):
  #     return DataLoader(self.mnist_test, batch_size=self.batch_size)

In [39]:
model = PLModel(channels=3, height=16, width=16, hidden_size=5, batch_size=8)

In [43]:
trainer = Trainer(
    accelerator="auto", # 使用accelerator来选择将要训练的硬件，可设置cpu、gpu、tpu以及自动选择
    max_epochs=50)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [30]:
# Process data
train_feature = torch.rand((16, 3, 16, 16))
train_label = torch.rand((16, 1))
torch_dataset = TensorDataset(train_feature, train_label)
loader = DataLoader(
    dataset=torch_dataset,
    batch_size=8,
    shuffle=True,
    num_workers=0,
)

In [40]:
trainer.fit(model, loader)

INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 7.8 K 
-------------------------------------
7.8 K     Trainable params
0         Non-trainable params
7.8 K     Total params
0.031     Total estimated model params size (MB)
/usr/local/lib/python3.9/dist-packages/pytorch_lightning/loops/fit_loop.py:280: PossibleUserWarning: The number of training batches (2) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.9/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=50` reached.


In [ ]:
# 全局变量
AVAIL_GPUS = min(1, torch.cuda.device_count())